# Análisis Exploratorio de Datos (EDA) - Dataset de Compras Black Sales

## 2a) EDA y preparación del dataset

En este análisis vamos a:
1. Explorar el dataset de compras realizadas durante un black sales
2. Realizar el pre-procesamiento necesario
3. Crear segmentos de clientes basados en el gasto
4. Preparar los datos para clasificación

In [ ]:
# Importar librerías necesarias
import time
import torch
import copy
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from collections import defaultdict

warnings.filterwarnings('ignore')

# Configuración de visualización
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Cargar el dataset
df = pd.read_csv('data/dataset_compras.csv')

# Información básica del dataset
print("Información general del dataset:")
print(f"Dimensiones: {df.shape}")
print(f"Filas: {df.shape[0]:,}")
print(f"Columnas: {df.shape[1]}")
print(f"\nTipos de datos:")
print(df.dtypes)
print(f"\nPrimeras 5 filas:")
df.head()

In [ ]:
print("Estadísticas descriptivas:")
print("Variables numéricas:")
print(df.describe())

print("\nInformación de columnas categóricas:")
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    print(f"{col}: {df[col].nunique()} valores únicos")
    print(f"  Valores: {df[col].unique()[:10]}")

In [ ]:
# Visualizaciones de las distribuciones
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Distribución de Purchase (variable objetivo)
axes[0, 0].hist(df['Purchase'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribución de Purchase')
axes[0, 0].set_xlabel('Purchase Amount')
axes[0, 0].set_ylabel('Frecuencia')

# Distribución por Género
gender_counts = df['Gender'].value_counts()
axes[0, 1].bar(gender_counts.index, gender_counts.values)
axes[0, 1].set_title('Distribución por Género')
axes[0, 1].set_xlabel('Género')
axes[0, 1].set_ylabel('Cantidad')

# Distribución por Edad
age_counts = df['Age'].value_counts()
axes[0, 2].bar(age_counts.index, age_counts.values, color='orange')
axes[0, 2].set_title('Distribución por Edad')
axes[0, 2].set_xlabel('Grupo de Edad')
axes[0, 2].set_ylabel('Cantidad')
axes[0, 2].tick_params(axis='x', rotation=45)

# Distribución por Estado Civil
marital_counts = df['Marital_Status'].value_counts()
axes[1, 0].bar(['Soltero', 'Casado'], marital_counts.values, color='green')
axes[1, 0].set_title('Distribución por Estado Civil')
axes[1, 0].set_xlabel('Estado Civil')
axes[1, 0].set_ylabel('Cantidad')

# Distribución por Categoría de Ciudad
city_counts = df['City_Category'].value_counts()
axes[1, 1].bar(city_counts.index, city_counts.values, color='red')
axes[1, 1].set_title('Distribución por Categoría de Ciudad')
axes[1, 1].set_xlabel('Categoría de Ciudad')
axes[1, 1].set_ylabel('Cantidad')

# Años en la ciudad actual
years_counts = df['Stay_In_Current_City_Years'].value_counts()
axes[1, 2].bar(years_counts.index, years_counts.values, color='purple')
axes[1, 2].set_title('Años en Ciudad Actual')
axes[1, 2].set_xlabel('Años')
axes[1, 2].set_ylabel('Cantidad')

plt.tight_layout()
plt.show()

In [ ]:
# Crear segmentos de clientes basados en Purchase
def crear_segmento(purchase):
    if purchase < 3000:
        return 'Casual'
    elif 3000 <= purchase < 8000:
        return 'Gama_media'
    elif 8000 <= purchase < 18000:
        return 'Gama_alta'
    else:
        return 'Premium'

# Aplicar la función para crear la nueva columna de segmentos
df['Segmento'] = df['Purchase'].apply(crear_segmento)

# Analizar la distribución de segmentos
segmento_counts = df['Segmento'].value_counts()
segmento_percentage = (segmento_counts / len(df)) * 100

print("Distribución de segmentos:")
for segmento, count in segmento_counts.items():
    pct = segmento_percentage[segmento]
    print(f"{segmento}: {count:,} clientes ({pct:.1f}%)")

# Visualización de la distribución de segmentos
plt.figure(figsize=(12, 6))

# Gráfico de barras
plt.subplot(1, 2, 1)
plt.bar(segmento_counts.index, segmento_counts.values, 
        color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
plt.title('Distribución de Segmentos de Clientes')
plt.xlabel('Segmento')
plt.ylabel('Cantidad de Clientes')
plt.xticks(rotation=45)

# Gráfico circular
plt.subplot(1, 2, 2)
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
plt.pie(segmento_counts.values, labels=segmento_counts.index, autopct='%1.1f%%', 
        colors=colors, startangle=90)
plt.title('Proporción de Segmentos de Clientes')

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de correlación
plt.style.use('default')
sns.set_palette("husl")
numeric_columns = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_columns].corr()

fig, axes = plt.subplots(2, 2, figsize=(20, 16))

# Heatmap de correlación completo
sns.heatmap(correlation_matrix, annot=True, cmap='RdBu_r', center=0, 
            fmt='.3f', square=True, ax=axes[0,0], cbar_kws={'shrink': .8})
axes[0,0].set_title('Matriz de Correlación Completa', fontsize=16, fontweight='bold', pad=20)
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].tick_params(axis='y', rotation=0)

# Correlaciones con Purchase
purchase_corr = correlation_matrix['Purchase'].abs().sort_values(ascending=False)
purchase_corr_display = purchase_corr[purchase_corr.index != 'Purchase'].head(10)

colors = ['red' if correlation_matrix['Purchase'][var] < 0 else 'blue' for var in purchase_corr_display.index]
bars = axes[0,1].barh(range(len(purchase_corr_display)), purchase_corr_display.values, color=colors, alpha=0.7)
axes[0,1].set_yticks(range(len(purchase_corr_display)))
axes[0,1].set_yticklabels(purchase_corr_display.index)
axes[0,1].set_xlabel('Correlación Absoluta con Purchase', fontweight='bold')
axes[0,1].set_title('Top 10 Variables Más Correlacionadas con Purchase', fontsize=16, fontweight='bold', pad=20)
axes[0,1].grid(True, alpha=0.3, axis='x')

for i, (bar, val) in enumerate(zip(bars, purchase_corr_display.values)):
    actual_corr = correlation_matrix['Purchase'][purchase_corr_display.index[i]]
    axes[0,1].text(val + 0.005, bar.get_y() + bar.get_height()/2, 
                   f'{actual_corr:.3f}', ha='left', va='center', fontweight='bold', fontsize=10)

# Distribución de valores de correlación
corr_values = correlation_matrix.values[np.triu_indices_from(correlation_matrix.values, k=1)]
axes[1,0].hist(corr_values, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[1,0].axvline(np.mean(corr_values), color='red', linestyle='--', 
                  label=f'Media: {np.mean(corr_values):.3f}')
axes[1,0].axvline(np.median(corr_values), color='orange', linestyle='--', 
                  label=f'Mediana: {np.median(corr_values):.3f}')
axes[1,0].set_xlabel('Valor de Correlación')
axes[1,0].set_ylabel('Frecuencia')
axes[1,0].set_title('Distribución de Coeficientes de Correlación', fontsize=16, fontweight='bold', pad=20)
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Correlación con Segmento
if 'Segmento' in df.columns:
    segmento_encoder = LabelEncoder()
    df_temp = df.copy()
    df_temp['Segmento_encoded'] = segmento_encoder.fit_transform(df['Segmento'])
    
    numeric_cols_with_segmento = df_temp.select_dtypes(include=[np.number]).columns
    corr_with_segmento = df_temp[numeric_cols_with_segmento].corr()['Segmento_encoded'].abs().sort_values(ascending=False)
    corr_with_segmento = corr_with_segmento[corr_with_segmento.index != 'Segmento_encoded'].head(8)
    
    colors_seg = ['green' if df_temp[numeric_cols_with_segmento].corr()['Segmento_encoded'][var] > 0 else 'orange' 
                  for var in corr_with_segmento.index]
    
    bars_seg = axes[1,1].barh(range(len(corr_with_segmento)), corr_with_segmento.values, 
                              color=colors_seg, alpha=0.7)
    axes[1,1].set_yticks(range(len(corr_with_segmento)))
    axes[1,1].set_yticklabels(corr_with_segmento.index)
    axes[1,1].set_xlabel('Correlación Absoluta con Segmento', fontweight='bold')
    axes[1,1].set_title('Variables Más Correlacionadas con Segmento de Cliente', fontsize=16, fontweight='bold', pad=20)
    axes[1,1].grid(True, alpha=0.3, axis='x')
    
    for i, (bar, val) in enumerate(zip(bars_seg, corr_with_segmento.values)):
        actual_corr_seg = df_temp[numeric_cols_with_segmento].corr()['Segmento_encoded'][corr_with_segmento.index[i]]
        axes[1,1].text(val + 0.005, bar.get_y() + bar.get_height()/2, 
                       f'{actual_corr_seg:.3f}', ha='left', va='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
print("Análisis de valores nulos:")

missing_analysis = pd.DataFrame({
    'Variable': df.columns,
    'Total_Nulos': df.isnull().sum(),
    'Porcentaje_Nulos': (df.isnull().sum() / len(df) * 100).round(2)
})
missing_analysis = missing_analysis.sort_values('Porcentaje_Nulos', ascending=False)

print("Distribución de valores nulos:")
print(missing_analysis[missing_analysis['Total_Nulos'] > 0])

vars_con_nulos = missing_analysis[missing_analysis['Total_Nulos'] > 0]['Variable'].tolist()

if len(vars_con_nulos) >= 2:
    df_nulos = df[vars_con_nulos].isnull()
    patrones_nulos = df_nulos.value_counts().head(4)
    print("\nPatrones de valores nulos:")
    for patron, count in patrones_nulos.items():
        print(f"  {patron}: {count:,} casos ({count/len(df)*100:.1f}%)")

for var_nulo in vars_con_nulos:
    if var_nulo not in ['User_ID', 'Product_ID']:
        total_productos = df['Product_ID'].nunique()
        productos_con_nulos = df[df[var_nulo].isnull()]['Product_ID'].nunique()
        pct_productos_afectados = (productos_con_nulos / total_productos) * 100
        
        if pct_productos_afectados > 30:
            tipo_nulo = "MNAR - Sistematico"
        elif pct_productos_afectados > 80:
            tipo_nulo = "MAR - Dependiente"
        else:
            tipo_nulo = "MCAR - Aleatorio"
        
        print(f"{var_nulo}: {pct_productos_afectados:.1f}% productos afectados - {tipo_nulo}")

print("Análisis completado")

### 2b) Modelo de deep leaning sin embeddings (1.5 puntos)
Entrenar un modelo de deep learning usando Pytorch que no utilice embeddings, **descartando el `product_id` y `user_id`**. Graficar las evoluciones por época de la función de costo y de métricas como el accuracy y el f1 score para train y validation. Mostrar una matriz de confusión absoluta y otra normalizada por fila. Explicar el proceso de iteracion utilizado para conseguir los resultados y justificar los resultados obtenidos.

Pueden usarse herramientas de regularización y prueba de hiperpametros para conseguir mejores resultados.

Un resultado aceptable sería al menos un 78% de accuracy y f1 score para el set de validation.

## Modelo DeepCustomerSegmentationNet

Vamos a aplicar técnicas avanzadas de regularización:
1. **Regularización L1 + L2 (Elastic Net)**
2. **Dropout mejorado con tasas progresivas**
3. **Learning Rate Scheduler avanzado (OneCycleLR)**
4. **Early Stopping más preciso**
5. **Gradient Clipping**

In [ ]:
# Separar características y variable objetivo
X = df.drop(['Purchase', 'Segmento', 'User_ID', 'Product_ID'], axis=1)
y = df['Segmento']

# Dividir dataset: 70% train, 15% val, 15% test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)

# Obtener índices correspondientes para cada conjunto
train_indices = X_train.index
val_indices = X_val.index
test_indices = X_test.index

# Calcular estadísticas del conjunto de entrenamiento
train_product_frequency = df.loc[train_indices, 'Product_ID'].value_counts().to_dict()

# Aplicar las estadísticas a todos los conjuntos
default_frequency = 1 

# Agregar Product_Frequency a cada conjunto
X_train = X_train.copy()
X_train['Product_Frequency'] = df.loc[train_indices, 'Product_ID'].map(train_product_frequency).fillna(default_frequency)

X_val = X_val.copy()
X_val['Product_Frequency'] = df.loc[val_indices, 'Product_ID'].map(train_product_frequency).fillna(default_frequency)

X_test = X_test.copy()
X_test['Product_Frequency'] = df.loc[test_indices, 'Product_ID'].map(train_product_frequency).fillna(default_frequency)

# Verificar productos no vistos
unseen_val = sum(df.loc[val_indices, 'Product_ID'].map(train_product_frequency).isna())
unseen_test = sum(df.loc[test_indices, 'Product_ID'].map(train_product_frequency).isna())
print(f"Productos no vistos en entrenamiento - Val: {unseen_val}, Test: {unseen_test}")

print(f"Train: {X_train.shape[0]} | Validation: {X_val.shape[0]} | Test: {X_test.shape[0]}")

# Definir columnas por tipo
numeric_cols = ['Marital_Status', 'Product_Category', 'Product_Subcategory_1', 'Product_Subcategory_2', 
                'Product_Frequency']
categorical_nominal = ['Gender', 'City_Category']
categorical_ordinal = ['Age', 'Stay_In_Current_City_Years']

# Configurar transformadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_cols),
        ('cat_nom', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_nominal),
        ('cat_ord', OrdinalEncoder(
            categories=[
                ['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'],
                ['1', '2', '3', '4', '5+']
            ],
            handle_unknown='use_encoded_value', 
            unknown_value=-1
        ), categorical_ordinal)
    ]
)

# Transformar datos
X_train_encoded = preprocessor.fit_transform(X_train)
X_val_encoded = preprocessor.transform(X_val)
X_test_encoded = preprocessor.transform(X_test)

# Imputación
imputer = SimpleImputer(strategy='constant', fill_value=-999)
X_train_imputed = imputer.fit_transform(X_train_encoded)
X_val_imputed = imputer.transform(X_val_encoded)
X_test_imputed = imputer.transform(X_test_encoded)

# Escalar datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Codificar target
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)
y_test_encoded = le.transform(y_test)

print(f"Shape final - Features: {X_train_scaled.shape[1]} | Classes: {len(le.classes_)}")
print(f"Clases: {le.classes_}")
print(f"Distribución train: {np.bincount(y_train_encoded)}")
print(f"Distribución val: {np.bincount(y_val_encoded)}")
print(f"Distribución test: {np.bincount(y_test_encoded)}")

In [ ]:
# Parámetros de configuración
EPOCHS = 50
BATCH_SIZE = 512
LEARNING_RATE = 0.001
MAX_LR = 0.01
WEIGHT_DECAY = 1e-4
L1_LAMBDA = 1e-5
PATIENCE = 10
MAX_GRAD_NORM = 1.0
LABEL_SMOOTHING = 0.1
NUM_WORKERS = 2

# Arquitectura del modelo
INPUT_SIZE = X_train_scaled.shape[1]
HIDDEN_LAYERS = [512, 256, 128, 64, 32]
NUM_CLASSES = 4
DROPOUT_RATES = [0.3, 0.4, 0.5, 0.6, 0.7]

class CustomerDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.FloatTensor(x)
        self.y = torch.LongTensor(y)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

class DeepCustomerSegmentationNet(nn.Module):
    def __init__(self, input_size=INPUT_SIZE, hidden_layers=HIDDEN_LAYERS, 
                 num_classes=NUM_CLASSES, dropout_rates=DROPOUT_RATES):
        super().__init__()
        
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        
        prev_size = input_size
        for i, (hidden_size, dropout_rate) in enumerate(zip(hidden_layers, dropout_rates)):
            self.layers.append(nn.Linear(prev_size, hidden_size))
            self.batch_norms.append(nn.BatchNorm1d(hidden_size))
            self.dropouts.append(nn.Dropout(dropout_rate))
            prev_size = hidden_size
        
        self.output_layer = nn.Linear(prev_size, num_classes)
        self.init_weights()
    
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        for layer, bn, dropout in zip(self.layers, self.batch_norms, self.dropouts):
            x = F.relu(bn(layer(x)))
            x = dropout(x)
        x = self.output_layer(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo utilizado: {device}")

train_dataset = CustomerDataset(X_train_scaled, y_train_encoded)
val_dataset = CustomerDataset(X_val_scaled, y_val_encoded)
test_dataset = CustomerDataset(X_test_scaled, y_test_encoded)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

model = DeepCustomerSegmentationNet().to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=PATIENCE, min_lr=1e-6)

print(f"Modelo creado con {sum(p.numel() for p in model.parameters())} parámetros")
print(f"Arquitectura: {INPUT_SIZE} -> {' -> '.join(map(str, HIDDEN_LAYERS))} -> {NUM_CLASSES}")

def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    return acc, f1, precision, recall

def train_epoch(model, loader, criterion, optimizer, device, l1_lambda=L1_LAMBDA):
    model.train()
    total_loss = 0
    all_preds, all_targets = [], []
    
    for batch_x, batch_y in loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss += l1_lambda * l1_norm
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=MAX_GRAD_NORM)
        optimizer.step()
        
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_targets.extend(batch_y.cpu().numpy())
    
    avg_loss = total_loss / len(loader)
    acc, f1, precision, recall = compute_metrics(all_targets, all_preds)
    return avg_loss, acc, f1, precision, recall

def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds, all_targets = [], []
    
    with torch.no_grad():
        for batch_x, batch_y in loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
            
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_targets.extend(batch_y.cpu().numpy())
    
    avg_loss = total_loss / len(loader)
    acc, f1, precision, recall = compute_metrics(all_targets, all_preds)
    return avg_loss, acc, f1, precision, recall

In [ ]:
epochs = EPOCHS
best_f1 = 0
patience = PATIENCE
patience_counter = 0
best_model = None

metrics = defaultdict(list)

print(f"Iniciando entrenamiento por {epochs} épocas")
print("Epoch | Train Loss | Train Acc | Train F1 | Val Loss | Val Acc | Val F1 | LR")
print("-" * 85)

for epoch in range(epochs):
    train_loss, train_acc, train_f1, train_prec, train_rec = train_epoch(
        model, train_loader, criterion, optimizer, device)
    
    val_loss, val_acc, val_f1, val_prec, val_rec = validate_epoch(
        model, val_loader, criterion, device)
    
    scheduler.step(val_f1)
    current_lr = optimizer.param_groups[0]['lr']
    
    metrics['train_loss'].append(train_loss)
    metrics['train_acc'].append(train_acc)
    metrics['train_f1'].append(train_f1)
    metrics['train_precision'].append(train_prec)
    metrics['train_recall'].append(train_rec)
    metrics['val_loss'].append(val_loss)
    metrics['val_acc'].append(val_acc)
    metrics['val_f1'].append(val_f1)
    metrics['val_precision'].append(val_prec)
    metrics['val_recall'].append(val_rec)
    metrics['learning_rate'].append(current_lr)
    
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_model = copy.deepcopy(model.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
    
    if (epoch + 1) % 2 == 0:
        print(f"{epoch+1:5d} | {train_loss:10.4f} | {train_acc:9.4f} | "
              f"{train_f1:8.4f} | {val_loss:8.4f} | {val_acc:7.4f} | {val_f1:6.4f} | {current_lr:.2e}")
    
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

model.load_state_dict(best_model)
test_loss, test_acc, test_f1, test_prec, test_rec = validate_epoch(
    model, test_loader, criterion, device)

torch.save(best_model, 'best_deep_model.pth')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].plot(metrics['train_loss'], label='Train Loss', color='blue', alpha=0.7)
axes[0, 0].plot(metrics['val_loss'], label='Validation Loss', color='red', alpha=0.7)
axes[0, 0].set_title('Loss durante el entrenamiento')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(metrics['train_acc'], label='Train Accuracy', color='blue', alpha=0.7)
axes[0, 1].plot(metrics['val_acc'], label='Validation Accuracy', color='red', alpha=0.7)
axes[0, 1].axhline(y=0.78, color='green', linestyle='--', alpha=0.8, label='Objetivo 78%')
axes[0, 1].set_title('Accuracy durante el entrenamiento')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(metrics['train_f1'], label='Train F1-Score', color='blue', alpha=0.7)
axes[1, 0].plot(metrics['val_f1'], label='Validation F1-Score', color='red', alpha=0.7)
axes[1, 0].axhline(y=0.78, color='green', linestyle='--', alpha=0.8, label='Objetivo 78%')
axes[1, 0].set_title('F1-Score durante el entrenamiento')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('F1-Score')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

final_metrics = ['Accuracy', 'F1-Score', 'Precision', 'Recall']
test_scores = [test_acc, test_f1, test_prec, test_rec]
colors = ['skyblue', 'lightgreen', 'lightcoral', 'lightsalmon']

bars = axes[1, 1].bar(final_metrics, test_scores, color=colors, alpha=0.8)
axes[1, 1].axhline(y=0.78, color='red', linestyle='--', alpha=0.8, label='Objetivo 78%')
axes[1, 1].set_title('Métricas finales en conjunto de test')
axes[1, 1].set_ylabel('Score')
axes[1, 1].set_ylim(0, 1)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

for bar, score in zip(bars, test_scores):
    axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    f'{score:.3f}', ha='center', va='bottom', fontweight='bold')

plt.suptitle('Entrenamiento del Modelo DeepCustomerSegmentationNet', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

model.eval()
with torch.no_grad():
    y_test_pred = []
    for batch_x, _ in test_loader:
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        y_test_pred.extend(preds)

print("Reporte de clasificación detallado:")
print(classification_report(y_test_encoded, y_test_pred, target_names=le.classes_, digits=4))

In [ ]:
model.eval()
with torch.no_grad():
    y_test_pred = []
    for batch_x, _ in test_loader:
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        y_test_pred.extend(preds)

y_test_pred = np.array(y_test_pred)
y_test_labels = le.inverse_transform(y_test_pred)
y_test_true_labels = le.inverse_transform(y_test_encoded)

results_df = pd.DataFrame({
    'Prediccion': y_test_labels,
    'Real': y_test_true_labels,
    'Correcto': y_test_labels == y_test_true_labels
})

test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_f1 = f1_score(y_test_encoded, y_test_pred, average='weighted')

print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1-Score: {test_f1:.4f}")
print(f"Distribución de predicciones:")
print(results_df['Prediccion'].value_counts().sort_index())
print(f"Primeras 20 predicciones:")
results_df.head(20)

### 2c) Modelo de deep leaning con embeddings (2 puntos)
Entrenar un modelo de deep learning usando Pytorch que utilice **2 capas de embeddings**, una para los productos y otra para los usuarios. Graficar las evoluciones por época de la función de costo y de métricas como el accuracy y el f1 score para train y validation. Mostrar una matriz de confusión absoluta y otra normalizada por fila. Explicar el proceso de iteracion utilizado para conseguir los resultados y justificar los resultados obtenidos. Justificar la cantidad de dimensiones usada para los embeddings. **Comparar contra el modelo sin embeddings** y explicar el porqué de los resultados.

Elegir la cantidad justa y necesaria de dimensiones para los embeddings y justificar el porqué.

Pueden usarse herramientas de regularización y prueba de hiperpametros para conseguir mejores resultados.

Un resultado aceptable sería al menos un 90% de accuracy y f1 score para el set de validation.

In [ ]:
# Preparación datos con embeddings
user_encoder = LabelEncoder()
product_encoder = LabelEncoder()

df_users = pd.Series(df['User_ID'].unique())
df_products = pd.Series(df['Product_ID'].unique())

user_encoder.fit(df_users)
product_encoder.fit(df_products)

X_embedding = df.drop(['Purchase', 'Segmento'], axis=1)
y_embedding = df['Segmento']

X_temp_emb, X_test_emb, y_temp_emb, y_test_emb = train_test_split(
    X_embedding, y_embedding, test_size=0.15, random_state=42, stratify=y_embedding)
X_train_emb, X_val_emb, y_train_emb, y_val_emb = train_test_split(
    X_temp_emb, y_temp_emb, test_size=0.176, random_state=42, stratify=y_temp_emb)

# Codificar usuarios y productos
X_train_emb = X_train_emb.copy()
X_train_emb['User_ID_encoded'] = user_encoder.transform(X_train_emb['User_ID'])
X_train_emb['Product_ID_encoded'] = product_encoder.transform(X_train_emb['Product_ID'])

X_val_emb = X_val_emb.copy()
X_val_emb['User_ID_encoded'] = user_encoder.transform(X_val_emb['User_ID'])
X_val_emb['Product_ID_encoded'] = product_encoder.transform(X_val_emb['Product_ID'])

X_test_emb = X_test_emb.copy()
X_test_emb['User_ID_encoded'] = user_encoder.transform(X_test_emb['User_ID'])
X_test_emb['Product_ID_encoded'] = product_encoder.transform(X_test_emb['Product_ID'])

# Agregar Product_Frequency
train_indices_emb = X_train_emb.index
val_indices_emb = X_val_emb.index
test_indices_emb = X_test_emb.index

train_product_frequency_emb = df.loc[train_indices_emb, 'Product_ID'].value_counts().to_dict()
default_frequency = 1

X_train_emb['Product_Frequency'] = df.loc[train_indices_emb, 'Product_ID'].map(train_product_frequency_emb).fillna(default_frequency)
X_val_emb['Product_Frequency'] = df.loc[val_indices_emb, 'Product_ID'].map(train_product_frequency_emb).fillna(default_frequency)
X_test_emb['Product_Frequency'] = df.loc[test_indices_emb, 'Product_ID'].map(train_product_frequency_emb).fillna(default_frequency)

# Preparar características 
categorical_cols = ['Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years']
numeric_embedding_cols = ['Marital_Status', 'Product_Category', 'Product_Subcategory_1', 'Product_Subcategory_2', 'Product_Frequency']

preprocessor_emb = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_embedding_cols),
        ('cat_nom', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), ['Gender', 'City_Category']),
        ('cat_ord', OrdinalEncoder(
            categories=[
                ['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'],
                ['1', '2', '3', '4', '5+']
            ],
            handle_unknown='use_encoded_value', 
            unknown_value=-1
        ), ['Age', 'Stay_In_Current_City_Years'])
    ]
)

X_train_cat = preprocessor_emb.fit_transform(X_train_emb[categorical_cols + numeric_embedding_cols])
X_val_cat = preprocessor_emb.transform(X_val_emb[categorical_cols + numeric_embedding_cols])
X_test_cat = preprocessor_emb.transform(X_test_emb[categorical_cols + numeric_embedding_cols])

imputer_emb = SimpleImputer(strategy='constant', fill_value=-999)
X_train_cat = imputer_emb.fit_transform(X_train_cat)
X_val_cat = imputer_emb.transform(X_val_cat)
X_test_cat = imputer_emb.transform(X_test_cat)

scaler_emb = StandardScaler()
X_train_cat_scaled = scaler_emb.fit_transform(X_train_cat)
X_val_cat_scaled = scaler_emb.transform(X_val_cat)
X_test_cat_scaled = scaler_emb.transform(X_test_cat)

y_train_emb_encoded = le.transform(y_train_emb)
y_val_emb_encoded = le.transform(y_val_emb)
y_test_emb_encoded = le.transform(y_test_emb)

NUM_USERS = len(user_encoder.classes_)
NUM_PRODUCTS = len(product_encoder.classes_)
USER_EMB_DIM = min(50, int(NUM_USERS**0.18))
PRODUCT_EMB_DIM = min(50, int(NUM_PRODUCTS**0.15))
CATEGORICAL_FEATURES = X_train_cat_scaled.shape[1]

print(f"Usuarios: {NUM_USERS} | Productos: {NUM_PRODUCTS}")
print(f"Embedding dims - User: {USER_EMB_DIM} | Product: {PRODUCT_EMB_DIM}")
print(f"Features categoricas: {CATEGORICAL_FEATURES}")
print(f"Shapes finales - Train: {X_train_cat_scaled.shape}, Val: {X_val_cat_scaled.shape}, Test: {X_test_cat_scaled.shape}")

In [ ]:
# Parámetros de configuración
EPOCHS = 50
BATCH_SIZE = 512
LEARNING_RATE = 0.001
MAX_LR = 0.01
WEIGHT_DECAY = 1e-4
L1_LAMBDA = 1e-3
PATIENCE = 3
MAX_GRAD_NORM = 1.0
LABEL_SMOOTHING = 0.1
NUM_WORKERS = 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo utilizado: {device}")

class EmbeddingDataset(Dataset):
    def __init__(self, user_ids, product_ids, categorical_features, targets):
        self.user_ids = torch.LongTensor(user_ids)
        self.product_ids = torch.LongTensor(product_ids)
        self.categorical_features = torch.FloatTensor(categorical_features)
        self.targets = torch.LongTensor(targets)
    
    def __len__(self):
        return len(self.user_ids)
    
    def __getitem__(self, idx):
        return (self.user_ids[idx], self.product_ids[idx], 
                self.categorical_features[idx], self.targets[idx])

class EmbeddingNet(nn.Module):
    def __init__(self, num_users, num_products, user_emb_dim, product_emb_dim, 
                 num_categorical_features, num_classes=4):
        super().__init__()
        
        self.user_embedding = nn.Embedding(num_users, user_emb_dim)
        self.product_embedding = nn.Embedding(num_products, product_emb_dim)
        
        total_input_size = user_emb_dim + product_emb_dim + num_categorical_features
        
        self.layers = nn.Sequential(
            nn.Linear(total_input_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.4),
            
            nn.Linear(512, 256),
            nn.ReLU(), 
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.6),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.6),
            
            nn.Linear(64, num_classes)
        )
        
        self.init_weights()
    
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Embedding):
                nn.init.xavier_uniform_(m.weight)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, user_ids, product_ids, categorical_features):
        user_emb = self.user_embedding(user_ids)
        product_emb = self.product_embedding(product_ids)
        
        x = torch.cat([user_emb, product_emb, categorical_features], dim=1)
        x = self.layers(x)
        return x

In [ ]:
train_dataset_emb = EmbeddingDataset(
    X_train_emb['User_ID_encoded'].values,
    X_train_emb['Product_ID_encoded'].values,
    X_train_cat_scaled,
    y_train_emb_encoded
)

val_dataset_emb = EmbeddingDataset(
    X_val_emb['User_ID_encoded'].values,
    X_val_emb['Product_ID_encoded'].values,
    X_val_cat_scaled,
    y_val_emb_encoded
)

test_dataset_emb = EmbeddingDataset(
    X_test_emb['User_ID_encoded'].values,
    X_test_emb['Product_ID_encoded'].values,
    X_test_cat_scaled,
    y_test_emb_encoded
)

train_loader_emb = DataLoader(train_dataset_emb, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader_emb = DataLoader(val_dataset_emb, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader_emb = DataLoader(test_dataset_emb, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

model_emb = EmbeddingNet(NUM_USERS, NUM_PRODUCTS, USER_EMB_DIM, PRODUCT_EMB_DIM, 
                        CATEGORICAL_FEATURES).to(device)

criterion_emb = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)
optimizer_emb = torch.optim.AdamW(model_emb.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler_emb = ReduceLROnPlateau(optimizer_emb, mode='max', factor=0.5, patience=PATIENCE, min_lr=1e-6)

print("Dataset con embeddings configurado:")
print(f"Train: {len(train_dataset_emb):,} muestras")
print(f"Validation: {len(val_dataset_emb):,} muestras") 
print(f"Test: {len(test_dataset_emb):,} muestras")
print(f"Batch size: {BATCH_SIZE}")
print(f"Total parámetros modelo: {sum(p.numel() for p in model_emb.parameters()):,}")

In [ ]:
def train_epoch_emb(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    all_preds, all_targets = [], []
    
    for user_ids, product_ids, categorical_features, targets in loader:
        user_ids = user_ids.to(device)
        product_ids = product_ids.to(device) 
        categorical_features = categorical_features.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(user_ids, product_ids, categorical_features)
        loss = criterion(outputs, targets)
        
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss += 1e-5 * l1_norm
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_targets.extend(targets.cpu().numpy())
    
    avg_loss = total_loss / len(loader)
    acc, f1, precision, recall = compute_metrics(all_targets, all_preds)
    return avg_loss, acc, f1, precision, recall

def validate_epoch_emb(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds, all_targets = [], []
    
    with torch.no_grad():
        for user_ids, product_ids, categorical_features, targets in loader:
            user_ids = user_ids.to(device)
            product_ids = product_ids.to(device)
            categorical_features = categorical_features.to(device)
            targets = targets.to(device)
            
            outputs = model(user_ids, product_ids, categorical_features)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_targets.extend(targets.cpu().numpy())
    
    avg_loss = total_loss / len(loader)
    acc, f1, precision, recall = compute_metrics(all_targets, all_preds)
    return avg_loss, acc, f1, precision, recall

In [ ]:
epochs_emb = EPOCHS
best_f1_emb = 0
patience_emb = PATIENCE
patience_counter_emb = 0
best_model_emb = None

metrics_emb = defaultdict(list)

print(f"Entrenamiento modelo con embeddings - {epochs_emb} épocas")
print("Epoch | Train Loss | Train Acc | Train F1 | Val Loss | Val Acc | Val F1")
print("-" * 75)

for epoch in range(epochs_emb):
    train_loss_emb, train_acc_emb, train_f1_emb, train_prec_emb, train_rec_emb = train_epoch_emb(
        model_emb, train_loader_emb, criterion_emb, optimizer_emb)
    
    val_loss_emb, val_acc_emb, val_f1_emb, val_prec_emb, val_rec_emb = validate_epoch_emb(
        model_emb, val_loader_emb, criterion_emb)
    
    scheduler_emb.step(val_f1_emb)
    
    metrics_emb['train_loss'].append(train_loss_emb)
    metrics_emb['train_acc'].append(train_acc_emb)
    metrics_emb['train_f1'].append(train_f1_emb)
    metrics_emb['val_loss'].append(val_loss_emb)
    metrics_emb['val_acc'].append(val_acc_emb)
    metrics_emb['val_f1'].append(val_f1_emb)
    
    if val_f1_emb > best_f1_emb:
        best_f1_emb = val_f1_emb
        best_model_emb = copy.deepcopy(model_emb.state_dict())
        patience_counter_emb = 0
    else:
        patience_counter_emb += 1
    
    if (epoch + 1) % 1 == 0:
        print(f"{epoch+1:5d} | {train_loss_emb:10.4f} | {train_acc_emb:9.4f} | "
              f"{train_f1_emb:8.4f} | {val_loss_emb:8.4f} | {val_acc_emb:7.4f} | {val_f1_emb:6.4f}")
    
    if patience_counter_emb >= patience_emb:
        print(f"Early stopping at epoch {epoch+1}")
        break

model_emb.load_state_dict(best_model_emb)
test_loss_emb, test_acc_emb, test_f1_emb, test_prec_emb, test_rec_emb = validate_epoch_emb(
    model_emb, test_loader_emb, criterion_emb)

torch.save(best_model_emb, 'best_improved_model.pth')

print(f"Resultados finales:")
print(f"Sin embeddings  - Test Acc: {test_acc:.4f} | Test F1: {test_f1:.4f}")
print(f"Con embeddings  - Test Acc: {test_acc_emb:.4f} | Test F1: {test_f1_emb:.4f}")
print(f"Mejora          - Acc: {test_acc_emb-test_acc:+.4f} | F1: {test_f1_emb-test_f1:+.4f}")

In [ ]:
# Visualizaciones y matriz de confusión
from sklearn.metrics import confusion_matrix

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

axes[0, 0].plot(metrics_emb['train_loss'], label='Train Loss', color='blue', alpha=0.7)
axes[0, 0].plot(metrics_emb['val_loss'], label='Validation Loss', color='red', alpha=0.7)
axes[0, 0].set_title('Loss - Modelo con Embeddings')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(metrics_emb['train_acc'], label='Train Accuracy', color='blue', alpha=0.7)
axes[0, 1].plot(metrics_emb['val_acc'], label='Validation Accuracy', color='red', alpha=0.7)
axes[0, 1].axhline(y=0.90, color='green', linestyle='--', alpha=0.8, label='Objetivo 90%')
axes[0, 1].set_title('Accuracy - Modelo con Embeddings')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[0, 2].plot(metrics_emb['train_f1'], label='Train F1-Score', color='blue', alpha=0.7)
axes[0, 2].plot(metrics_emb['val_f1'], label='Validation F1-Score', color='red', alpha=0.7)
axes[0, 2].axhline(y=0.90, color='green', linestyle='--', alpha=0.8, label='Objetivo 90%')
axes[0, 2].set_title('F1-Score - Modelo con Embeddings')
axes[0, 2].set_xlabel('Epoch')
axes[0, 2].set_ylabel('F1-Score')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

model_emb.eval()
with torch.no_grad():
    y_test_pred_emb = []
    for user_ids, product_ids, categorical_features, _ in test_loader_emb:
        user_ids = user_ids.to(device)
        product_ids = product_ids.to(device)
        categorical_features = categorical_features.to(device)
        outputs = model_emb(user_ids, product_ids, categorical_features)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        y_test_pred_emb.extend(preds)

cm_absolute = confusion_matrix(y_test_emb_encoded, y_test_pred_emb)
cm_normalized = confusion_matrix(y_test_emb_encoded, y_test_pred_emb, normalize='true')

im1 = axes[1, 0].imshow(cm_absolute, interpolation='nearest', cmap='Blues')
axes[1, 0].set_title('Matriz de Confusión Absoluta')
axes[1, 0].set_xlabel('Predicción')
axes[1, 0].set_ylabel('Real')
axes[1, 0].set_xticks(range(4))
axes[1, 0].set_yticks(range(4))
axes[1, 0].set_xticklabels(le.classes_)
axes[1, 0].set_yticklabels(le.classes_)

for i in range(4):
    for j in range(4):
        axes[1, 0].text(j, i, str(cm_absolute[i, j]), ha='center', va='center', 
                       color='white' if cm_absolute[i, j] > cm_absolute.max()/2 else 'black')

im2 = axes[1, 1].imshow(cm_normalized, interpolation='nearest', cmap='Blues')
axes[1, 1].set_title('Matriz de Confusión Normalizada')
axes[1, 1].set_xlabel('Predicción')
axes[1, 1].set_ylabel('Real')
axes[1, 1].set_xticks(range(4))
axes[1, 1].set_yticks(range(4))
axes[1, 1].set_xticklabels(le.classes_)
axes[1, 1].set_yticklabels(le.classes_)

for i in range(4):
    for j in range(4):
        axes[1, 1].text(j, i, f'{cm_normalized[i, j]:.3f}', ha='center', va='center',
                       color='white' if cm_normalized[i, j] > 0.5 else 'black')

comparison_metrics = ['Accuracy', 'F1-Score', 'Precision', 'Recall']
sin_emb_scores = [test_acc, test_f1, test_prec, test_rec]
con_emb_scores = [test_acc_emb, test_f1_emb, test_prec_emb, test_rec_emb]

x_pos = np.arange(len(comparison_metrics))
width = 0.35

bars1 = axes[1, 2].bar(x_pos - width/2, sin_emb_scores, width, 
                      label='Sin Embeddings', alpha=0.8, color='lightcoral')
bars2 = axes[1, 2].bar(x_pos + width/2, con_emb_scores, width,
                      label='Con Embeddings', alpha=0.8, color='lightblue')

axes[1, 2].axhline(y=0.90, color='green', linestyle='--', alpha=0.8, label='Objetivo 90%')
axes[1, 2].set_title('Comparación de Modelos')
axes[1, 2].set_xlabel('Métricas')
axes[1, 2].set_ylabel('Score')
axes[1, 2].set_xticks(x_pos)
axes[1, 2].set_xticklabels(comparison_metrics)
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

for bars, scores in [(bars1, sin_emb_scores), (bars2, con_emb_scores)]:
    for bar, score in zip(bars, scores):
        axes[1, 2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                       f'{score:.3f}', ha='center', va='bottom', fontsize=9)

plt.suptitle('Modelo con Embeddings - Análisis Completo', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("Reporte de clasificación - Modelo con Embeddings:")
print(classification_report(y_test_emb_encoded, y_test_pred_emb, target_names=le.classes_, digits=4))

print(f"Justificación de dimensiones de embeddings:")
print(f"- User embedding: {USER_EMB_DIM} dimensiones para {NUM_USERS} usuarios")
print(f"  Fórmula: min(50, usuarios^0.18) = min(50, {NUM_USERS}^0.18) ≈ {int(NUM_USERS**0.18)}")
print(f"- Product embedding: {PRODUCT_EMB_DIM} dimensiones para {NUM_PRODUCTS} productos") 
print(f"  Fórmula: min(50, productos^0.15) = min(50, {NUM_PRODUCTS}^0.15) ≈ {int(NUM_PRODUCTS**0.15)}")
print(f"Razón: Dimensiones suficientes para capturar patrones sin sobreajuste")

print(f"Explicación de resultados:")
mejora_acc = test_acc_emb - test_acc
mejora_f1 = test_f1_emb - test_f1
if mejora_acc > 0 and mejora_f1 > 0:
    print(f"Los embeddings mejoran el rendimiento:")
    print(f"  - Capturan patrones latentes usuario-producto")
    print(f"  - Reducen la dimensionalidad categórica")
    print(f"  - Mejor representación de entidades")
else:
    print(f"Embeddings no mejoran significativamente:")
    print(f"  - Dataset puede ser demasiado pequeño")
    print(f"  - Patrones ya capturados por features categóricas")
    print(f"  - Necesita más regularización o ajuste")

## Justificación de Dimensiones de Embeddings

### Dimensiones Elegidas:
- **User Embedding**: 4 dimensiones para 7,432 usuarios
- **Product Embedding**: 3 dimensiones para 2,455 productos

### Fórmula utilizada:
```
USER_EMB_DIM = min(50, int(NUM_USERS**0.18))
PRODUCT_EMB_DIM = min(50, int(NUM_PRODUCTS**0.15))
```

### Justificación:
1. **Evitar overfitting**: Dimensiones pequeñas previenen memorización
2. **Eficiencia computacional**: Menos parámetros = entrenamiento más rápido
3. **Generalización**: Fuerza al modelo a aprender patrones generales
4. **Regla empírica**: Raíz cuarta del número de entidades es un buen punto de partida

## Herramientas de Regularización Utilizadas

### ¿Por qué usar regularización?
- **Prevenir overfitting**: Evita que el modelo memorice los datos de entrenamiento
- **Mejorar generalización**: Permite mejor rendimiento en datos no vistos
- **Estabilizar entrenamiento**: Reduce la varianza en el aprendizaje

### Técnicas aplicadas:

1. **Dropout (0.4-0.6)**: Desactiva neuronas aleatoriamente durante entrenamiento
2. **L1 Regularization (1e-5)**: Penaliza pesos grandes, promueve sparsity
3. **Weight Decay (1e-4)**: Regularización L2 en el optimizador
4. **Label Smoothing (0.1)**: Suaviza las etiquetas objetivo
5. **Gradient Clipping (1.0)**: Previene gradientes explosivos
6. **Early Stopping (patience=3)**: Detiene entrenamiento cuando no hay mejora
7. **Batch Normalization**: Normaliza activaciones entre capas

### Resultados:
- Sin regularización: Overfitting severo (100% accuracy)
- Con regularización: Modelo más estable y generalizable

### 2d) Encontrar usuarios similares (1 punto)
Para el modelo del punto 2c) implementar una función que reciba un ID de usuario y sugiera **n** cantidad de usuarios que tuvieron un comportamiento de compras similar. También se debe mostrar el grado o porcentaje de similitud de cada usuario que retorne la función.

In [ ]:
def encontrar_usuarios_similares(user_id, n=5, metrica='coseno'):
    """
    Encuentra usuarios con comportamiento similar basado en embeddings.
    
    Args:
        user_id: ID del usuario de referencia
        n: Número de usuarios similares a retornar
        metrica: 'coseno', 'producto_punto' o 'euclidiana'
    
    Returns:
        DataFrame con usuarios similares y porcentajes de similitud
    """
    if user_id not in user_encoder.classes_:
        return f"Usuario {user_id} no encontrado en el dataset"
    
    user_idx = user_encoder.transform([user_id])[0]
    
    model_emb.eval()
    with torch.no_grad():
        user_embeddings = model_emb.user_embedding.weight.cpu().numpy()
        user_ref_embedding = user_embeddings[user_idx].reshape(1, -1)
        
        if metrica == 'coseno':
            similarities = cosine_similarity(user_ref_embedding, user_embeddings)[0]
        elif metrica == 'producto_punto':
            similarities = np.dot(user_embeddings, user_ref_embedding.T).flatten()
        elif metrica == 'euclidiana':
            distances = euclidean_distances(user_ref_embedding, user_embeddings)[0]
            similarities = 1 / (1 + distances)
        else:
            raise ValueError("Métrica no válida")
    
    user_ids_decoded = user_encoder.classes_
    results_df = pd.DataFrame({
        'User_ID': user_ids_decoded,
        'Similitud': similarities,
        'Porcentaje': (similarities * 100).round(2) if metrica == 'coseno' else 
                     ((similarities - similarities.min()) / (similarities.max() - similarities.min()) * 100).round(2)
    })
    
    results_df = results_df[results_df['User_ID'] != user_id]
    results_df = results_df.sort_values('Similitud', ascending=False).head(n)
    
    user_stats = []
    for similar_user in results_df['User_ID'].values:
        user_data = df[df['User_ID'] == similar_user]
        stats = {
            'Compras': len(user_data),
            'Gasto_Medio': user_data['Purchase'].mean().round(2),
            'Productos': user_data['Product_ID'].nunique(),
            'Segmento': user_data['Segmento'].mode().iloc[0] if len(user_data) > 0 else 'N/A'
        }
        user_stats.append(stats)
    
    stats_df = pd.DataFrame(user_stats)
    return pd.concat([results_df.reset_index(drop=True), stats_df], axis=1)

def analizar_usuario(user_id):
    """Retorna estadísticas básicas del usuario."""
    user_data = df[df['User_ID'] == user_id]
    return {
        'compras': len(user_data),
        'gasto_medio': user_data['Purchase'].mean().round(2),
        'productos': user_data['Product_ID'].nunique(),
        'segmento': user_data['Segmento'].mode().iloc[0]
    }

def comparar_usuarios(user_ref, usuarios_similares_df):
    """Compara usuario de referencia con usuarios similares."""
    ref_stats = analizar_usuario(user_ref)
    
    print(f"Usuario de referencia: {user_ref}")
    print(f"Compras: {ref_stats['compras']}, Gasto medio: {ref_stats['gasto_medio']}")
    print(f"Productos únicos: {ref_stats['productos']}, Segmento: {ref_stats['segmento']}")
    print()
    
    print("Usuarios similares:")
    for idx, row in usuarios_similares_df.iterrows():
        print(f"{row['User_ID']} - Similitud: {row['Porcentaje']:.1f}% - "
              f"Compras: {row['Compras']} - Gasto: {row['Gasto_Medio']} - "
              f"Segmento: {row['Segmento']}")

def evaluar_embeddings():
    """Evalúa la calidad de los embeddings generados."""
    with torch.no_grad():
        user_embeddings = model_emb.user_embedding.weight.cpu().numpy()
        product_embeddings = model_emb.product_embedding.weight.cpu().numpy()
    
    similarities_matrix = cosine_similarity(user_embeddings)
    np.fill_diagonal(similarities_matrix, np.nan)
    
    print("Estadísticas de embeddings:")
    print(f"Usuarios: {user_embeddings.shape}, Productos: {product_embeddings.shape}")
    print(f"Similitud promedio: {np.nanmean(similarities_matrix):.3f}")
    print(f"Rango similitud: [{np.nanmin(similarities_matrix):.3f}, {np.nanmax(similarities_matrix):.3f}]")

# Ejemplo de uso
usuario_muestra = np.random.choice(df['User_ID'].unique())
print(f"Ejemplo con usuario: {usuario_muestra}")

usuarios_similares = encontrar_usuarios_similares(usuario_muestra, n=5)
print(f"Resultados similitud coseno:")
print(usuarios_similares[['User_ID', 'Porcentaje', 'Compras', 'Segmento']])

print(f"Comparación detallada:")
comparar_usuarios(usuario_muestra, usuarios_similares)

print(f"Evaluación de embeddings:")
evaluar_embeddings()